In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import copy

In [71]:
# Run this function if you do not have a proper **price.csv** file
def get_prices_dataframe():
    prices_list = []

    for i in range(len(list_of_coins)):
        # Read through CSV files
        lowercase_coin_name = str.lower(list_of_coins[i])
        temp_df = pd.read_csv('data/' + lowercase_coin_name + '-historical-day-data-all-tokeninsight.csv') 

        # Select 'Date', 'Price' columns and reverse order the dataframe from previous to current date
        data = temp_df[['Date', 'Price']] 
        data = data.iloc[::-1].reset_index(drop=True)   

        # Rename 'Price' columns with the coin's name
        current_coin = list_of_coins[i]
        data.rename(columns={"Price": current_coin}, inplace=True)

        # Append dataframes to corresponding lists
        prices_list.append(data)

    # Making a deepcopy of the first dataframe to avoid mutation of the original list
    prices = copy.deepcopy(prices_list[0])

    # Setting 'Date' column values as index
    prices.set_index('Date', inplace=True)

    for i in range(len(list_of_coins) - 1):
        prices_temp_df = prices_list[i+1].set_index('Date')
        prices = prices.join(prices_temp_df)

    # Resetting index back to normal
    prices.reset_index(inplace=True)

    # Formatting 'Data' column
    prices['Date'] = pd.to_datetime(prices['Date'])
    prices['Date'] = prices['Date'].dt.strftime('%Y-%m-%d')

    return prices

# Calculating returns from prices dataframe
def get_returns_dataframe():
    returns = prices.iloc[:,:]
    
    for i in range(len(list_of_coins)):
        coin_name = list_of_coins[i]
        returns[coin_name] = returns.iloc[:,i+1].ffill().pct_change(1)

    return returns

# Calculating volatility from returns dataframe
def get_volatility_dataframe():
    returns = get_returns_dataframe()
    volatility = returns.iloc[:,:]
    
    for i in range(len(list_of_coins)):
        coin_name = list_of_coins[i]
        volatility[coin_name]= volatility.iloc[:, i+1].rolling(window=365).std() * np.sqrt(365)
    return volatility

def get_subsystem_position(capital, volatility_target, instrument, instrument_forecast):

    # Calculating variables for volatility targeting
    annul_volatility_target = capital * volatility_target
    daily_cash_volatility_target = annul_volatility_target / np.sqrt(365)

    # Calculating price volatility - expected daily standart deviation of instrument returns
    price_volatility = 0.013
    
    instrument_block = 1

    # Calculating block_value based on the latest price of the given instrument
    price = prices[instrument].iloc[-1]
    block_value = price * 0.01

    
    # Calculating the subsystem position
    instrument_value_volatility = (price_volatility * 100) * block_value
    volatility_scalar = daily_cash_volatility_target / instrument_value_volatility

    # Finally getting the subsystem
    subsystem_position = instrument_forecast * volatility_scalar * 0.1
    
    return subsystem_position

In [70]:
# Reading prices.csv file
prices = pd.read_csv('prices.csv')
list_of_coins = ['BITCOIN', 'ETHEREUM', 'BINANCE-COIN', 'SOLANA']
# Parameters
capital = 10000
volatility_target = 0.25

get_subsystem_position(capital, volatility_target, 'SOLANA', 10)

2.11159


np.float64(61.97035440002719)